In [ ]:
# %%
import google.generativeai as genai
import time
import threading
import json
import os
from pathlib import Path
from tqdm import tqdm
import tempfile
import chardet
import zipfile
import shutil
import concurrent.futures
import re
import openai

# %%
# Initialize the thread lock
lock = threading.Lock()

# %%
# Configuration for Gemini
gemini_model = None
def run_gemini_query(prompt, history, completion_tokens, temp, model):
    global gemini_model
    try:
        if not gemini_model:
            genai.configure(api_key="")            
            gemini_model = genai.GenerativeModel(model)

        safety_settings = [
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
        ]

        response = gemini_model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                candidate_count=1,
                max_output_tokens=completion_tokens,
                temperature=temp
            ),
            safety_settings=safety_settings
        )

        input_toks = response.usage_metadata.prompt_token_count
        output_toks = response.usage_metadata.candidates_token_count
        cost_usd = input_toks / 1_000_000 * 1.25 + output_toks / 1_000_000 * 5

        # Update costs.json with locking
        with lock:
            if os.path.exists('gemini_costs.json'):
                with open('gemini_costs.json', 'r') as f:
                    costs = json.load(f)
            else:
                costs = {}
            costs[model] = costs.get(model, 0) + cost_usd
            with open('gemini_costs.json', 'w') as f:
                json.dump(costs, f)

        inference = response.text.strip() if response.text else None

        if inference:
            return inference
        else:
            print('Error: message is empty')
            time.sleep(5)

    except Exception as e:
        print("Request failed.")
        print(e)
        time.sleep(5)

    return None

# %%
# Configuration for OpenAI
api_key = ""
openai_client = openai.OpenAI(api_key=api_key)

def run_openai_query(prompt, history, completion_tokens, temp, model, openai_client):
    response = None
    try:
        messages = history + [{"role": "user", "content": prompt}]
        
        response = openai_client.chat.completions.create(
            model=model,
            temperature=temp,
            max_tokens=completion_tokens,
            messages=messages,
        )
        content = response.choices[0].message.content.strip()
            
        if content:
            return content
        else:
            print('Error: message is empty')
            time.sleep(5)

    except Exception as e:
        print("Request failed.")
        print(e)
        time.sleep(5)

    return None

# %%
# Function to parse scenes from text
def parse_scenes_from_text(text: str):
    try:
        scene_pattern = re.compile(r'## SCENE \d+:\n(.+?)(?=(?:## SCENE \d+:|$))', re.DOTALL)
        scenes = scene_pattern.findall(text)
        return [scene.strip() for scene in scenes] if scenes else []
    except Exception as e:
        return []

# %%
# Selection prompt template
selection_prompt = """
[TEXT START]
<TEXT>
[TEXT END]

Your task is to examine the above **public domain** work for evocative, interesting, well-written scenes.

Instructions:

- Choose 3 long scenes of approx 1000-1500 words, and output them in their entirety.
- You must also repair paragraphs by restoring sentences within a paragraph that are broken by a newline.
- Double newline between paragraphs.
- Do not include chapter names.

Output in this format:

## THOUGHT PROCESS:

<thought process for the scene selection>

## SCENE 1:

<the text of approx 1000-1500 words>

## SCENE 2:

<the text of approx 1000-1500 words>

## SCENE 3:

<the text of approx 1000-1500 words>


--

Output precisely in this format. Do not add any additional commentary or explanations.
"""

# %%
# Function to detect file encoding
def detect_encoding(file_path: Path):
    try:
        with open(file_path, 'rb') as f:
            raw_data = f.read(100000)  # Read first 100KB for detection
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        confidence = result['confidence']
        if encoding:
            encoding_lower = encoding.lower()
            if encoding_lower in ['ascii', 'charmap']:
                return 'latin-1'
            if confidence >= 0.5:
                return encoding
        return 'utf-8'  # Fallback encoding
    except Exception as e:
        print(f"Error detecting encoding for '{file_path}': {e}. Falling back to 'utf-8'.")
        return 'utf-8'  # Fallback encoding

# %%
# Worker function to process a single file
def process_file(fn, base_path, results_path, selection_prompt):
    # Check if file already processed
    with lock:
        if os.path.exists(results_path):
            with open(results_path, 'r') as f:
                scenes = json.load(f)
        else:
            scenes = {}
        if fn in scenes:
            print(f"Skipping '{fn}': already processed.")
            return

    # Check if file exists
    zip_path = os.path.join(base_path, fn)
    if not os.path.exists(zip_path):
        print(f"Skipping '{fn}': file does not exist.")
        return
    
    with tempfile.TemporaryDirectory() as tmpdirname:
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(tmpdirname)
        except zipfile.BadZipFile:
            print(f"Skipping '{zip_path}': Not a valid zip file.")
            return
        except Exception as e:
            print(f"Error extracting '{zip_path}': {e}")
            return

        extracted_files = list(Path(tmpdirname).rglob("*"))
        text_files = [f for f in extracted_files if f.is_file() and f.suffix.lower() in ['.txt', '.md', '.text']]

        if len(text_files) != 1:
            print(f"Skipping '{zip_path}': Expected 1 text file, found {len(text_files)}.")
            return

        text_file = text_files[0]

        # Detect encoding
        encoding = detect_encoding(text_file)
        try:
            with open(text_file, 'r', encoding=encoding, errors='replace') as f:
                content = f.read()
        except Exception as e:
            print(f"Error reading '{text_file}': {e}")
            return

        if len(content) < 10000:
            print(f"Skipping '{fn}': content too small!")
            return
        
        if content.find('End of the Project Gutenberg'):
            content = content[:content.find('End of the Project Gutenberg')].strip()
        else:
            print('!! not found')

        max_seg_length = 400000
        # Split into 4 even segments, ensuring max length
        #segment_length = max_seg_length
        #segments = [content[i:i+segment_length] for i in range(0, len(content), segment_length)]
        #segments = segments[:4]  # Limit to first 4 segments
        segments = [content[i:i+len(content)//4][:max_seg_length] for i in range(0, len(content), len(content)//4)]
        segments = segments[:4]

        # Collect scenes from all segments
        all_scenes = []
        for i, segment in enumerate(segments):
            prompt = selection_prompt
            if i == 0:
                prompt += "\nThe first scene you select should be the very start of the book. The other two can be selected from anywhere."

            prompt = prompt.replace('<TEXT>', segment)

            result = run_gemini_query(prompt, [], 16000, 0, "gemini-1.5-pro-002")

            #print('-'*10)
            #print(result)
            #print('-'*10)

            if result:
                this_scenes = parse_scenes_from_text(result)
                print(len(this_scenes))
                #if not scenes:
                #    print('!!')

                all_scenes.extend(this_scenes)
            else:
                print(f"Failed to get scenes for '{fn}' segment {i+1}.")

            

        # Update scenes.json with locking
        with lock:
            if os.path.exists(results_path):
                with open(results_path, 'r') as f:
                    scenes = json.load(f)
            else:
                scenes = {}
            if fn not in scenes:
                scenes[fn] = []
            scenes[fn].extend(all_scenes)
            with open(results_path, 'w') as f:
                json.dump(scenes, f, indent=2)


In [5]:
import json

# %%
# Main function to orchestrate multithreading
def main():
    #filenames_path = 'data/fiction_book_selections.json'
    #filenames_path = 'data/fantasy_book_selections.json'
    #filenames_path = 'data/scifi_book_selections.json'
    filenames_path = 'data/romance_book_selections.json'
    base_path = "/mnt/i/gutenberg_processed/"
    #results_path = "scenes.json"
    #results_path = "scenes_fantasy.json"
    #results_path = "scenes_scifi.json"
    results_path = "scenes_romance.json"
    num_threads = 8

    # Load filenames
    with open(filenames_path, 'r') as f:
        filenames = json.load(f)

    #print(len(filenames))
    
    with open('data/fiction_book_selections.json', 'r') as f:
        already_selected = json.load(f)

    

    filenames = [fn for fn in filenames if fn not in already_selected] 
    #print(len(filenames))

    with open('data/scifi_book_selections.json', 'r') as f:
        already_selected = json.load(f)

    

    filenames = [fn for fn in filenames if fn not in already_selected] 
    #print(len(filenames))

    

    # Initialize tqdm
    progress = tqdm(total=len(filenames), desc="Processing files")

    # Define a helper to update tqdm
    def update_progress(future):
        progress.update(1)
        exception = future.exception()
        if exception:
            print(f"Error occurred: {exception}")

    # Use ThreadPoolExecutor for multithreading
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []
        for fn in filenames:
            future = executor.submit(process_file, fn, base_path, results_path, selection_prompt)
            future.add_done_callback(update_progress)
            futures.append(future)

        # Wait for all futures to complete
        concurrent.futures.wait(futures)

    progress.close()
    print("Processing completed.")


In [6]:
main()

Processing files:   0%|          | 0/57 [00:00<?, ?it/s]

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


Processing files:   2%|▏         | 1/57 [07:17<6:48:38, 437.83s/it]

3
3
3
3
3
3
3


Processing files:   4%|▎         | 2/57 [09:28<3:55:33, 256.97s/it]

...

3


Processing files:  98%|█████████▊| 56/57 [1:15:41<00:57, 57.70s/it]

3


Processing files: 100%|██████████| 57/57 [1:16:55<00:00, 80.97s/it]

3
Processing completed.
